In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from skimage.segmentation import clear_border
import pytesseract
from pytesseract import Output
import cv2


In [2]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
cv2.namedWindow("output", cv2.WINDOW_NORMAL)  

In [63]:
img=cv2.imread(r"C:\Users\dhruv\Downloads\g2.png")

In [64]:
cv2.imshow("img",img)

In [10]:
img=cv2.resize(img,(400,600))

In [11]:
orig=img.copy()

In [12]:
ratio=img.shape[0]/300.0

In [16]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
cv2.imshow('edg', img_grey) 

In [17]:
img_blur = cv2.GaussianBlur(img_grey,(7,7),0)
cv2.imshow('edg', img_blur) 

In [18]:
img_edge=cv2.Canny(img_blur,28,48)
cv2.imshow('edg', img_edge) 

In [19]:
cnts,hier=cv2.findContours(img_edge,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts=sorted(cnts,key=cv2.contourArea,reverse=True)

In [20]:
for c in cnts:
    cnt_peri= cv2.arcLength(c,True)
    c_approx=cv2.approxPolyDP(c,0.02*cnt_peri,True)
    
    if len(c_approx)==4:
        selected_window = c_approx
        break
        

In [30]:
cv2.drawContours(img, [selected_window], -1, (0, 255, 0), 3)
cv2.imshow("Game Boy Screen", img)
cv2.waitKey(0)

-1

In [21]:
def find_sheet_end_point(window):
    window = window.reshape(4,2)
    new_window = np.zeros((4,2),dtype=np.float32)
    
    add = window.sum(axis=1)
    diff = np.diff(window,axis=1)
    
    new_window[0] = window[np.argmin(add)]
    new_window[1] = window[np.argmin(diff)]
    new_window[2] = window[np.argmax(add)]
    new_window[3] = window[np.argmax(diff)]
    
    #new_window*=ratio
    return new_window

In [22]:
c_approx = find_sheet_end_point(selected_window)


In [23]:
(tl, tr, br, bl) = c_approx



In [24]:
widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

In [25]:
heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

In [26]:
maxWidth = max(int(widthA), int(widthB))
maxHeight = max(int(heightA), int(heightB))


In [27]:
dst = np.array([
	[0, 0],
	[maxWidth - 1, 0],
	[maxWidth - 1, maxHeight - 1],
	[0, maxHeight - 1]], dtype = "float32")


In [28]:
M = cv2.getPerspectiveTransform(c_approx, dst)
warp = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))

In [29]:
cv2.imshow("persp",warp)

In [20]:
orig=warp.copy()

In [65]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  
cv2.imshow('Grayscale', img_grey) 

In [66]:
ret, img_thresh = cv2.threshold(img_grey, 135, 255, cv2.THRESH_BINARY) 

cv2.imshow('Binary Threshold', img_thresh) 

In [76]:
img_blur = cv2.GaussianBlur(img_grey,(7,7),0)
cv2.imshow('Blur', img_blur) 

In [77]:
kernel = np.ones((5,5),np.uint8)

In [78]:
img_er = cv2.erode(img_blur,kernel,iterations = 1)
cv2.imshow("Erosion",img_er)



In [87]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 4))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))


In [88]:
tophat = cv2.morphologyEx(img_er, cv2.MORPH_BLACKHAT, rectKernel)
cv2.imshow("morph",tophat)

In [89]:
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,ksize=5)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")
cv2.imshow("grad",gradX)

In [90]:
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv2.imshow("r",thresh)

In [74]:
cnts = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#cnts.sort(key=lambda x:get_contour_precedence(x, img.shape[1]))
locs = []

In [75]:
for (i, c) in enumerate(cnts):
	# compute the bounding box of the contour, then use the
	# bounding box coordinates to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	if (w > 30) and (h > 20)and(h<50):
         locs.append((x, y, w, h))
print(locs)
       


[]


In [62]:
for l in locs:
    x, y, w, h = l
    cv2.rectangle(orig, (x-5, y-5), (x+w+5,y+h+5), (50, 50, 255), 2)
    
#for i in range(len(locs)):
    #img = cv2.putText(img, str(i), cv2.boundingRect(locs[i])[:2], cv2.FONT_HERSHEY_COMPLEX, 1, [125])
cv2.imshow('img', orig)
cv2.waitKey(0)

-1

In [32]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [46]:


d = pytesseract.image_to_data(img, output_type=Output.DICT)
n_boxes = len(d['level'])
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])    
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow('img',img)

In [43]:
tessdata_dir_config = "/usr/share/tesseract-ocr/4.00/tessdata"
extracted_text = pytesseract.image_to_string(img,config=tessdata_dir_config)

In [44]:
extracted_text = pytesseract.image_to_string(img)

In [153]:
cv2.imshow("img",img_thresh)

In [45]:
print(extracted_text)

Text Message
Today 15:46

Its Emma. | tried to call you but
signal bad. | been taken to
hospital after having a fall this
morning. If possible can you do
me a quick favour and text me x



In [ ]:
kernel = np.ones((7,7),np.uint8)

In [ ]:
img_er = cv2.erode(img_blur,kernel,iterations = 1)
cv2.imshow("Erosion",img_er)

In [ ]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 6))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

In [ ]:
tophat = cv2.morphologyEx(img_er, cv2.MORPH_BLACKHAT, rectKernel)
#cv2.imshow("morph",tophat)

In [ ]:
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,ksize=5)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")
#cv2.imshow("grad",gradX)

In [ ]:
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv2.imshow("r",thresh)

In [ ]:
# find contours in the thresholded image, then initialize the
# list of digit locations
cnts = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#cnts.sort(key=lambda x:get_contour_precedence(x, img.shape[1]))
locs = []

In [ ]:
# loop over the contours
for (i, c) in enumerate(cnts):
	# compute the bounding box of the contour, then use the
	# bounding box coordinates to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	if (w > 70) and (h > 20)and(h<50):
		# append the bounding box region of the digits group
		# to our locations list
		locs.append((x, y, w, h))

In [ ]:
for l in locs:
    x, y, w, h = l
    cv2.rectangle(warp, (x-5, y-5), (x+w+5,y+h+5), (50, 50, 255), 2)
    
#for i in range(len(locs)):
    #img = cv2.putText(img, str(i), cv2.boundingRect(locs[i])[:2], cv2.FONT_HERSHEY_COMPLEX, 1, [125])
cv2.imshow('img', warp)
cv2.waitKey(0)

In [ ]:
pip install imutils
